# You suck at shell scripting!

You do! You're aweful! You are as bad at shell scripting
as Helen Zille is at Twitter. However clever that 
140 character bash one liner seemed at the time,
you know that you regretted it once it became an
integral part of your production build pipeline. This
talk will show you how to write maintainable
Command Line Interfaces with `click` and `pathlib`
that won't leave you trying to rewrite your git commit
history after the next production server outage.


In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/U_X5uR7VC4M" frameborder="0" allowfullscreen></iframe>


## Aim

  * demonstrate
      * pathlib
      * click
  * worked example for file deduplication

## shell scripts

  * Provide some examples of badly written shell scripts
  * Why I've resisted using Python for shell scripting.
      * os.path

## pathlib

Path handling the right way

In [ ]:
from pathlib import Path
p = Path('.')
p

In [ ]:
print(str(p))
print(p.absolute())

In [ ]:
p = p.absolute()
print(p.as_posix())

In [ ]:
print(p.as_uri())

In [ ]:
p.is_dir()

In [ ]:
p.is_file()

In [ ]:
p.parent

In [ ]:
p.relative_to(p.parent)

In [ ]:
# subdirectories
[x for x in p.iterdir() if x.is_dir()]

In [ ]:
# files
[x for x in p.iterdir() if x.is_file()]

In [ ]:
list(p.rglob('*.ipynb'))

In [ ]:
p = Path('.') / 'newdir'
p

In [ ]:
p.exists()

In [ ]:
p.mkdir()

In [ ]:
p.exists()

In [ ]:
q = p / 'newfile.txt'
q

In [ ]:
with q.open('wt') as f:
    f.write('The quick brown fox jumped over the lazy dog.')

In [ ]:
q.exists() and q.is_file()

In [ ]:
q.read_text()

In [ ]:
q.unlink()

In [ ]:
q.exists()

In [ ]:
p.rmdir()

In [ ]:
p.exists()

# CLIs

  * 40 years of bad branding as **Command Line Intefaces**
  * Who knows what that is?
  * Rebrand as _"Chatbot Like Interface"_

## Click 

  * **"Command Line Interface Creation Kit"** or 
  * **"Chatbot Like Interface Creation Kit"**?

Beautiful CLI APIs
  * automatic help page generation
  * parameter validation
  * arbitrary nesting of commands
  * supports lazy loading of subcommands at runtime

In [ ]:
%%HTML
<iframe width="1024" height="400" src="http://click.pocoo.org/5/"></iframe>

### To cover
  
* arguments vs options
* parameter types

### Basic example

```python
# %load hello.py
import click

@click.command()
@click.option('--count', default=1, help='Number of greetings.')
@click.option('--name', prompt='Your name', help='The person to greet.')
def hello(count, name):
    """Simple program that greets NAME for a total of COUNT times."""
    for x in range(count):
        click.echo('Hello {}!'.format(name))

if __name__=='__main__':
    hello()
```

In [ ]:
!python hello.py --name Tobias --count 3

## Extended example

In [ ]:
# %load ysass.py

In [ ]:
Path('file1.txt').write_text('Hello world!')
Path('file2.txt').write_text('Hello world!')

In [ ]:
!python ysass.py

In [ ]:
!python ysass.py digests show

In [ ]:
!python ysass.py digests --help

In [ ]:
!python ysass.py digests -p file1.txt show

In [ ]:
!python ysass.py digests -h sha1 show

In [ ]:
!python ysass.py digests -h sha256 show

## setuptools integration

```python
# %load setup.py
from setuptools import setup

setup(
    name='ysass',
    version='0.1',
    py_modules=['hello', 'ysass'],
    install_requires=[
        'Click',
    ],
    entry_points='''
        [console_scripts]
        hello=hello:hello
        ysass=ysass:cli
    ''',
)
```

In [ ]:
!pip install -e .

In [ ]:
!hello --name Tobias --count 2

In [ ]:
!ysass --help

In [ ]:
!ysass digests show

In [ ]:
!ysass digests duplicates show